# Relational Databases
Relational databases are a huge topic and worthy of several semesters of college courses to fully grasp. In this notebook, we will take a glance at the sample database based on a music store called the [chinook database.](http://www.sqlitetutorial.net/sqlite-sample-database/) If you would like to view the database in sqlite, the open source [DB Browser for SQLite](http://sqlitebrowser.org/) is a great option.

One of the best things to do when first diving into a proper relational database is to study a diagram of the **data model** to better understand how tables are related. 

![chinook](http://www.sqlitetutorial.net/wp-content/uploads/2015/11/sqlite-sample-database-color.jpg)

This diagram tells us a great deal about all the relations.
1. Each rectangle is a table
1. The table name is the top orange line
1. All the column names follow under the table name
1. The data type of each column is also listed - usually either numeric, character or date
1. All tables have at least one column with a key symbol denoting either a primary key when there is one or a composite primary key when more than 1.
1. All the primary keys have lines joining other tables (or themselves)
1. The columns that these lines join to are called foreign keys
1. Just as the lines are connecting to the tables you will see different symbols - representing the type of relationship
1. One to one, One to many and many to many are the major relationship types.  
1. The 'crows feet' represent many, the single bar represents 1 and the occasional 'O' reprsents an 0 or an optional relationship.
1. Good relations typically are of the many to many variety

### Begin Answering Questions
We will assume that the data is in reasonably good shape since it is a sample relational database and begin answering questions.

In [2]:
import pandas as pd
import numpy as np

pd.options.display.max_columns = 40

### Check that relationship matches the diagram

We will manually inspect whether some of the relationships really do match the diagram. Take a look at the relationship between **albums** and **artists**. Both a one-to-one and a many-to-one relationship exist depending on which way the relationship is going. Each row in the **albums** table matches with at least one and at most one row in the **artists** table. The two vertical bars indicate the least and greatest number of matches between those tables.

Going the other way, each row in the **artists** table matches with any number of rows in the **albums** table. The crows feet alert us of this.

Let's check this relationship out by merging the two tables. If there really is a one to one relationship between **albums** and **artists** then an **inner join** should not change the number of rows.

In [3]:
albums = pd.read_csv('data/chinook/albums.csv')
artists = pd.read_csv('data/chinook/artists.csv')

In [4]:
albums.head()

,AlbumId,Title,ArtistId
0,1,For Those About To Rock We Salute You,1
1,2,Balls to the Wall,2
2,3,Restless and Wild,2
3,4,Let There Be Rock,1
4,5,Big Ones,3


In [5]:
artists.head()

,ArtistId,Name
0,1,AC/DC
1,2,Accept
2,3,Aerosmith
3,4,Alanis Morissette
4,5,Alice In Chains


In [6]:
albums.shape, artists.shape

((347, 3), (275, 2))

In [7]:
# if there is no on column merge will align on the column with the same name.
albums_artist = albums.merge(artists, on='ArtistId')

albums_artist.head(10)

,AlbumId,Title,ArtistId,Name
0,1,For Those About To Rock We Salute You,1,AC/DC
1,4,Let There Be Rock,1,AC/DC
2,2,Balls to the Wall,2,Accept
3,3,Restless and Wild,2,Accept
4,5,Big Ones,3,Aerosmith
5,6,Jagged Little Pill,4,Alanis Morissette
6,7,Facelift,5,Alice In Chains
7,8,Warner 25 Anos,6,Antônio Carlos Jobim
8,34,Chill: Brazil (Disc 2),6,Antônio Carlos Jobim
9,9,Plays Metallica By Four Cellos,7,Apocalyptica


In [8]:
# number of rows stayed the same! It is a one to one relationship
albums_artist.shape

(347, 4)

### Checking the other direction

A one to many relationship exists in the other direction (artist to album). Said in plain english: An artist can make many albums but an album can be made by only one artist. It is clar from the first few rows that many of the artists repeat and have indeed made more than one album.

### Checking if any artists have not made any albums

Could it be possible that there are artists in the database that have not made any albums? To do this we would need to do a left join between artists and check if there are any missing values for album.

In [9]:
artists_albums_left = artists.merge(albums, how='left')
artists_albums_left.head(10)

,ArtistId,Name,AlbumId,Title
0,1,AC/DC,1.0,For Those About To Rock We Salute You
1,1,AC/DC,4.0,Let There Be Rock
2,2,Accept,2.0,Balls to the Wall
3,2,Accept,3.0,Restless and Wild
4,3,Aerosmith,5.0,Big Ones
5,4,Alanis Morissette,6.0,Jagged Little Pill
6,5,Alice In Chains,7.0,Facelift
7,6,Antônio Carlos Jobim,8.0,Warner 25 Anos
8,6,Antônio Carlos Jobim,34.0,Chill: Brazil (Disc 2)
9,7,Apocalyptica,9.0,Plays Metallica By Four Cellos


In [10]:
# filter for null values in albumid
# wow, there are a lot of artists that do not have an album in the album table
artists_albums_left[artists_albums_left.AlbumId.isnull()].head(10)

,ArtistId,Name,AlbumId,Title
50,25,Milton Nascimento & Bebeto,NaN,NaN
51,26,Azymuth,NaN,NaN
55,28,João Gilberto,NaN,NaN
56,29,Bebel Gilberto,NaN,NaN
57,30,Jorge Vercilo,NaN,NaN
58,31,Baby Consuelo,NaN,NaN
59,32,Ney Matogrosso,NaN,NaN
60,33,Luiz Melodia,NaN,NaN
61,34,Nando Reis,NaN,NaN
62,35,Pedro Luís & A Parede,NaN,NaN


In [11]:
# specifically check that artistid 26 - Azymuth is not in albums table
26 in list(albums.ArtistId)

False

### Following the connections

Often you will have to join more than two dataframes to get a desired result. For instance, what if we wanted to know which genre was every album. The **genres** and **albums** table do not directly connect. We will have to go join the intermediary table **tracks** to map genre all the way back to album.

In [12]:
# First connect albums to tracs
tracks = pd.read_csv('data/chinook/tracks.csv')

In [13]:
# since each album has many tracks the size of the dataframe will greatly expand
album_track = albums.merge(tracks)

album_track.shape

(3503, 11)

In [14]:
# most of these columns are unnecessary so lets only keep genre id
album_genre = album_track[albums.columns.tolist() + ['GenreId']]

album_genre.head(10)

,AlbumId,Title,ArtistId,GenreId
0,1,For Those About To Rock We Salute You,1,1
1,1,For Those About To Rock We Salute You,1,1
2,1,For Those About To Rock We Salute You,1,1
3,1,For Those About To Rock We Salute You,1,1
4,1,For Those About To Rock We Salute You,1,1
5,1,For Those About To Rock We Salute You,1,1
6,1,For Those About To Rock We Salute You,1,1
7,1,For Those About To Rock We Salute You,1,1
8,1,For Those About To Rock We Salute You,1,1
9,1,For Those About To Rock We Salute You,1,1


In [15]:
# since there were many tracks per album lots of duplicates were create
# lets remove the duplicates with drop_duplicates
album_genre = album_genre.drop_duplicates()

In [16]:
# and finally join with the genre table
genre = pd.read_csv('data/chinook/genres.csv')

genre.head(10)

,GenreId,Name
0,1,Rock
1,2,Jazz
2,3,Metal
3,4,Alternative & Punk
4,5,Rock And Roll
5,6,Blues
6,7,Latin
7,8,Reggae
8,9,Pop
9,10,Soundtrack


In [17]:
album_genre_final = album_genre.merge(genre)

album_genre_final.head(10)

,AlbumId,Title,ArtistId,GenreId,Name
0,1,For Those About To Rock We Salute You,1,1,Rock
1,2,Balls to the Wall,2,1,Rock
2,3,Restless and Wild,2,1,Rock
3,4,Let There Be Rock,1,1,Rock
4,5,Big Ones,3,1,Rock
5,6,Jagged Little Pill,4,1,Rock
6,7,Facelift,5,1,Rock
7,10,Audioslave,8,1,Rock
8,30,BBC Sessions [Disc 1] [Live],22,1,Rock
9,31,Bongo Fury,23,1,Rock


### Does each Album have one genre?

There is no way to tell from the diagram whether each album has one genre as the relationship spans three tables with a many to many relationship in between. Lets check this by seeing if the **`AlbumId`** column repeats.

In [18]:
album_genre_final.AlbumId.value_counts().head()

141    3
227    3
109    2
228    2
261    2
Name: AlbumId, dtype: int64

In [19]:
# lets look at one of these albums that has more than one genre
# it makes sense that a greatest hits album can span multiple genres

album_genre_final[album_genre_final.AlbumId == 141]

,AlbumId,Title,ArtistId,GenreId,Name
60,141,Greatest Hits,100,1,Rock
151,141,Greatest Hits,100,3,Metal
237,141,Greatest Hits,100,8,Reggae


# Problem Set

You will have to import the tables as needed

In [1]:
import numpy as np
import pandas as pd

### Problem 1
<span  style="color:green; font-size:16px">How many media types does each track have? Answer this by looking at the data diagram and then programmatically.</span>

In [4]:
tracks = pd.read_csv('data/chinook/tracks.csv')
media_types = pd.read_csv('data/chinook/media_types.csv')

In [9]:
a = tracks.merge(media_types, on='MediaTypeId').groupby('TrackId')['MediaTypeId'].count()
print(a.min())
print(a.max())

1
1


### Problem 2
<span  style="color:green; font-size:16px">Which track has sold the most copies?</span>

In [10]:
invoice_items = pd.read_csv('data/chinook/invoice_items.csv')

In [27]:
tracks_invoice = tracks.merge(invoice_items, on='TrackId')
tracks_invoice.groupby('Name')['Quantity'].sum().sort_values(ascending=False).head(5)

Name
The Trooper                62
Hallowed Be Thy Name       58
The Number Of The Beast    44
Stairway To Heaven         43
Good Golly Miss Molly      42
Name: Quantity, dtype: int64

In [28]:
tracks_invoice.groupby('TrackId')['Quantity'].sum().sort_values(ascending=False).head(5)

TrackId
2031    38
184     36
1888    36
925     36
2352    36
Name: Quantity, dtype: int64

### Problem 3
<span  style="color:green; font-size:16px">Which playlist has the most tracks?</span>

In [20]:
playlists = pd.read_csv('data/chinook/playlists.csv')
playlist_track = pd.read_csv('data/chinook/playlist_track.csv')

In [22]:
tracks_playlists = tracks.merge(playlist_track, on='TrackId')
tracks_playlists_full = tracks_playlists.merge(playlists, on='PlaylistId', suffixes=('_track', '_playlist'))

In [26]:
tracks_playlists_full.groupby('Name_playlist')['Name_playlist'].size().sort_values(ascending=False)

Name_playlist
Music                         6580
90’s Music                    1477
TV Shows                       426
Classical                       75
Brazilian Music                 39
Heavy Metal Classic             26
Classical 101 - The Basics      25
Classical 101 - Next Steps      25
Classical 101 - Deep Cuts       25
Grunge                          15
On-The-Go 1                      1
Music Videos                     1
dtype: int64

### Problem 4
<span  style="color:green; font-size:16px">Which playlist, that has at least 15 tracks has on average the most expensive tracks?</span>

In [33]:
# tracks_playlists_invoice = tracks_playlists_full.merge(invoice_items, on='TrackId', suffixes=('_tracks', '_invoice'))

In [42]:
t = tracks_playlists_full.groupby('Name_playlist')['Name_playlist', 'UnitPrice'].filter(lambda s: len(s)>=15)
tracks_playlists_full.groupby('Name_playlist')['UnitPrice'].mean().sort_values(ascending=False)

Name_playlist
On-The-Go 1                   1.250000
Brazilian Music               1.021795
TV Shows                      1.001455
Music                         0.997860
Classical 101 - Deep Cuts     0.995600
Grunge                        0.995333
90’s Music                    0.993521
Heavy Metal Classic           0.987692
Classical                     0.952933
Classical 101 - The Basics    0.937200
Classical 101 - Next Steps    0.926000
Music Videos                  0.750000
Name: UnitPrice, dtype: float64

### Problem 5: Advanced
<span  style="color:green; font-size:16px">Find the most sold genre per country.</span>

In [47]:
invoices = pd.read_csv('data/chinook/invoices.csv')
customers = pd.read_csv('data/chinook/customers.csv')
genres = pd.read_csv('data/chinook/genres.csv')

In [48]:
tra_invitem_inv_cust = tracks.merge(invoice_items, on='TrackId')
tra_invitem_inv_cust = tra_invitem_inv_cust.merge(invoices, on='InvoiceId')
tra_invitem_inv_cust = tra_invitem_inv_cust.merge(customers, on='CustomerId')
tra_invitem_inv_cust = tra_invitem_inv_cust.merge(genres, on='GenreId', suffixes=('_track', '_genre'))

In [63]:
t = tra_invitem_inv_cust.groupby(['Country', 'Name_genre'], as_index=False)['Quantity'].sum()
t = t.sort_values(['Country', 'Quantity'], ascending=[True, False])
t.groupby('Country').first()

,Name_genre,Quantity
Country,,
Argentina,Alternative & Punk,106
Australia,Rock,226
Austria,Rock,97
Belgium,Rock,194
Brazil,Rock,769
Canada,Rock,1086
Chile,Latin,98
Czech Republic,Rock,275
Denmark,Rock,214


### Problem 6
<span  style="color:green; font-size:16px">Find the name and email of each employee's boss. When the left table and right table have different column names for the joining key use the arguments **`left_on`** and **`right_on`**. Make use of the suffix arguments to better label the merged data. Be sure to include employees that don't have bosses. This is called a recursive relationship.</span>

In [65]:
employees = pd.read_csv('data/chinook/employees.csv')
list(employees.columns)

['EmployeeId',
 'LastName',
 'FirstName',
 'Title',
 'ReportsTo',
 'BirthDate',
 'HireDate',
 'Address',
 'City',
 'State',
 'Country',
 'PostalCode',
 'Phone',
 'Fax',
 'Email']

In [69]:
employee_boss = employees.merge(employees, left_on='EmployeeId', right_on='ReportsTo', 
                                suffixes=('_employee', '_boss'), how='left')

In [72]:
employee_boss[['LastName_employee', 'FirstName_employee', 
               'LastName_boss', 'FirstName_boss', 'Email_boss']].head(10)

,LastName_employee,FirstName_employee,LastName_boss,FirstName_boss,Email_boss
0,Adams,Andrew,Edwards,Nancy,nancy@chinookcorp.com
1,Adams,Andrew,Mitchell,Michael,michael@chinookcorp.com
2,Edwards,Nancy,Peacock,Jane,jane@chinookcorp.com
3,Edwards,Nancy,Park,Margaret,margaret@chinookcorp.com
4,Edwards,Nancy,Johnson,Steve,steve@chinookcorp.com
5,Peacock,Jane,NaN,NaN,NaN
6,Park,Margaret,NaN,NaN,NaN
7,Johnson,Steve,NaN,NaN,NaN
8,Mitchell,Michael,King,Robert,robert@chinookcorp.com
9,Mitchell,Michael,Callahan,Laura,laura@chinookcorp.com


### Problem 7
<span  style="color:green; font-size:16px">Which artists have the longest tracks on average?</span>

In [75]:
artists = pd.read_csv('data/chinook/artists.csv')
albums = pd.read_csv('data/chinook/albums.csv')

In [77]:
art_alb_tra = artists.merge(albums, on='ArtistId')
art_alb_tra = art_alb_tra.merge(tracks, on='AlbumId', suffixes=('_artist', '_track'))

In [83]:
art_alb_tra.groupby('Name_artist')['Milliseconds'].mean().sort_values(ascending=False).head()

Name_artist
Battlestar Galactica (Classic)    2925574
Battlestar Galactica              2770464
Heroes                            2599142
Lost                              2589984
Aquaman                           2484567
Name: Milliseconds, dtype: int64

In [116]:
def millisec_to_min_sec(t):
    t = t / 1000
    minutes = t // 60
    seconds = t % 60
    return '''{:.0f}'{:.0f}'' '''.format(minutes, seconds)

In [118]:
art_alb_tra.groupby('Name_artist')['Milliseconds'].mean().sort_values(ascending=False).map(millisec_to_min_sec).head(10)

Name_artist
Battlestar Galactica (Classic)               48'46'' 
Battlestar Galactica                         46'10'' 
Heroes                                       43'19'' 
Lost                                         43'10'' 
Aquaman                                      41'25'' 
The Office                                   23'34'' 
Leonard Bernstein & New York Philharmonic     9'57'' 
Scholars Baroque Ensemble                     9'42'' 
Terry Bozzio, Tony Levin & Steve Stevens      9'36'' 
Adrian Leaper & Doreen de Feis                9'27'' 
Name: Milliseconds, dtype: object